# Import Dependencies

In [46]:
import numpy
import pandas as pd
import sqlite3
from sqlalchemy import create_engine,func
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# Use Pandas to convert CSV to SQLITE

In [57]:
df = pd.read_csv('../data/pothole_datasd.csv')
df = df[['service_request_id', 'lat', 'lng', 'status', 'case_origin', 'council_district','comm_plan_name']]
connection = sqlite3.connect('potholes.sqlite')
df.to_sql('potholes', con=connection, if_exists='replace')

# SQLAlchemy Prep: Create custom Python Class
Note: Tried using `Base = automap_base()` method but reflection was not working.

Source: [https://github.com/davidjaimes/sqlalchemy-challenge/blob/master/climate.ipynb](https://github.com/davidjaimes/sqlalchemy-challenge/blob/master/climate.ipynb)

In [58]:
Base = declarative_base()

class Potholes(Base):
    __tablename__ = 'potholes'
    service_request_id = Column(Integer, primary_key=True)
    lat =  Column(Float)
    lng =  Column(Float)
    status =  Column(String)
    case_origin = Column(String)
    council_district = Column(Integer, primary_key = True)
    comm_plan_name = Column(String)

# Create database connection and Begin Queries

In [16]:
engine = create_engine('sqlite:///potholes.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)

### Example 1: Flatten SQL query.

In [17]:
# Query all latitudes and print first element.
latitudes = session.query(Potholes.lat).all()
print(latitudes[0])

# Problem: Value is inside a SET.
# Solution: Use NumPy to turn to array and then flatten.
latitudes = numpy.array(latitudes).flatten()
print(latitudes[0])

(32.831881,)
32.831881


### Example 2: Filter SQL query.

In [18]:
new_requests = session.query(Potholes).filter(Potholes.status == 'In Process')
new_requests.count()

921

In [19]:
new_requests = session.query(Potholes).filter(Potholes.status == 'Closed')
new_requests.count()

65122

In [20]:
new_requests = session.query(Potholes).filter(Potholes.status == 'New')
new_requests.count()

2

In [21]:
new_requests = session.query(Potholes).filter(Potholes.status == 'Referred')
new_requests.count()

1851

In [22]:
new_requests = session.query(Potholes).filter(Potholes.case_origin == 'Phone')
new_requests.count()

6378

In [23]:
new_requests = session.query(Potholes).filter(Potholes.case_origin == 'Web')
new_requests.count()

22592

In [24]:
new_requests = session.query(Potholes).filter(Potholes.case_origin == 'Email')
new_requests.count()

324

In [29]:
# Design a query to show how many districts are available in this dataset?
def GetDistricts():
    results = session.query(Potholes.council_district).group_by(Potholes.council_district).count()
    print(f"{results} Districts total.")
    
GetDistricts()

10 Districts total.


In [35]:
# What are the most active districts? 
# List the districts and the counts in descending order.
active_districts = session.query(Potholes.council_district, func.count(Potholes.council_district))\
.group_by(Potholes.council_district).order_by(func.count(Potholes.council_district).desc()).all()
active_districts

[(6.0, 12154),
 (2.0, 11400),
 (3.0, 10060),
 (7.0, 8091),
 (1.0, 6403),
 (9.0, 6249),
 (5.0, 5986),
 (4.0, 4071),
 (8.0, 3237),
 (None, 0)]

In [59]:
# What are the most active referrals? 
# List the districts and the counts in descending order.
active_referrals = session.query(Potholes.case_origin, func.count(Potholes.case_origin))\
.group_by(Potholes.case_origin).order_by(func.count(Potholes.case_origin).desc()).all()
active_referrals

[('Mobile', 38586),
 ('Web', 22592),
 ('Phone', 6378),
 ('Email', 324),
 ('Route Slip', 7),
 ('Letter', 3),
 ('Walk-In', 2),
 ('Station 38 Incident Reports', 2),
 ('Garage Run', 1),
 ('Crew/Self Generated', 1)]

In [63]:
# What are the most active 'common places'? 
# List the common places and the counts in descending order.
active_places = session.query(Potholes.comm_plan_name, func.count(Potholes.comm_plan_name))\
.group_by(Potholes.comm_plan_name).order_by(func.count(Potholes.comm_plan_name).desc()).all()
active_places

[('Clairemont Mesa', 7105),
 ('Mira Mesa', 5060),
 ('Navajo', 3244),
 ('Uptown', 3034),
 ('Kearny Mesa', 2952),
 ('Rancho Bernardo', 2589),
 ('North Park', 2513),
 ('La Jolla', 2493),
 ('Pacific Beach', 2461),
 ('Downtown', 2400),
 ('Mid-City:City Heights', 2237),
 ('Peninsula', 2066),
 ('University', 1935),
 ('Mid-City:Eastern Area', 1828),
 ('College Area', 1776),
 ('Encanto Neighborhoods', 1727),
 ('Mission Valley', 1599),
 ('Midway-Pacific Highway', 1535),
 ('Linda Vista', 1453),
 ('Skyline-Paradise Hills', 1347),
 ('Southeastern San Diego', 1093),
 ('Mid-City:Kensington-Talmadge', 1061),
 ('Tierrasanta', 1032),
 ('Torrey Pines', 1027),
 ('Otay Mesa-Nestor', 980),
 ('Mission Bay Park', 925),
 ('Carmel Mountain Ranch', 892),
 ('Rancho Penasquitos', 860),
 ('Serra Mesa', 840),
 ('Mid-City:Normal Heights', 830),
 ('Scripps Miramar Ranch', 745),
 ('Greater Golden Hill', 685),
 ('Ocean Beach', 616),
 ('Carmel Valley', 581),
 ('Otay Mesa', 535),
 ('Reserve', 469),
 ('Balboa Park', 420),


In [54]:
#Group the districts by status on service request id
district_status = df.groupby(["council_district", "status"]).agg({"service_request_id":"count"}).reset_index()
district_status

,council_district,status,service_request_id
0,1.0,Closed,6011
1,1.0,In Process,94
2,1.0,New,1
3,1.0,Referred,297
4,2.0,Closed,10958
5,2.0,In Process,70
6,2.0,Referred,372
7,3.0,Closed,9549
8,3.0,In Process,107
9,3.0,Referred,404


In [53]:
#Group the referral 'case_origin' by district on service request id
district_referral = df.groupby(["council_district", "case_origin"]).agg({"service_request_id":"count"}).reset_index()
district_referral

,council_district,case_origin,service_request_id
0,1.0,Email,28
1,1.0,Mobile,3361
2,1.0,Phone,690
3,1.0,Web,2324
4,2.0,Crew/Self Generated,1
5,2.0,Email,118
6,2.0,Mobile,5925
7,2.0,Phone,1201
8,2.0,Web,4155
9,3.0,Email,32


In [48]:
#test to run sql
pd.read_sql("select * from potholes", con = engine)

,index,service_request_id,lat,lng,status,case_origin,council_district
0,0,80508,32.831881,-117.231233,Closed,Phone,2.0
1,1,80510,32.716021,-117.159052,Closed,Web,3.0
2,2,80518,32.760286,-117.104798,Closed,Phone,9.0
3,3,80531,32.955916,-117.252152,Closed,Phone,1.0
4,4,80535,32.710295,-117.155278,Closed,Web,3.0
...,...,...,...,...,...,...,...
67891,67891,2917671,32.696246,-117.027972,In Process,Mobile,4.0
67892,67892,2917697,32.745593,-117.127221,In Process,Mobile,3.0
67893,67893,2917713,32.820203,-117.177989,In Process,Mobile,6.0
67894,67894,2917718,32.774806,-117.153850,In Process,Phone,7.0
